In [1]:
import pandas as pd
import numpy as np

In [3]:
cols = ['Distance_from_hole','Par_Value','Cat']
data = pd.concat([pd.read_csv('./../data/%d.csv' % year)[cols] for year in range(2003,2017)])

In [4]:
def make_specific_cats(distance,cat,par):
    if cat=='Tee Box':
        if par==3:
            return 'Tee-3'
        else:
            return 'Tee-45'
    elif cat=='Bunker' or cat=='Other':
        return cat
    elif cat=='Green' or cat=='Fringe':
        if distance<5:
            return 'Green-0'
        elif distance<10:
            return 'Green-5'
        elif distance<20:
            return 'Green-10'
        else:
            return 'Green-20'
    elif cat=='Fairway':
        if distance<300:
            return 'Fairway-0'
        elif distance<540:
            return 'Fairway-300'
        else:
            return 'Fairway-540'
    elif cat=='Primary Rough' or cat=='Intermediate Rough':
        if distance<90:
            return 'Rough-0'
        elif distance<375:
            return 'Rough-90'
        else:
            return 'Rough-375'

In [5]:
cats = pd.Series([make_specific_cats(dist,cat,par) 
                  for dist,cat,par in zip(data.Distance_from_hole,data.Cat,data.Par_Value)])

In [6]:
cats.value_counts()/float(len(cats))

Green-0        0.199993
Tee-45         0.197405
Green-20       0.096400
Fairway-300    0.081263
Green-10       0.073506
Green-5        0.062079
Tee-3          0.058461
Rough-375      0.049375
Fairway-540    0.045494
Fairway-0      0.042789
Bunker         0.035739
Rough-0        0.034069
Rough-90       0.018598
Other          0.004828
dtype: float64